In [1]:
/**
 * Solve an SQP problem with both inequality and equality constraints using an active-set solver.
 *
 * example 15.4 in Andreas Antoniou, Wu-Sheng Lu
 *
 * @author Haksun Li
 */

%use s2
import dev.nm.solver.multivariate.constrained.general.sqp.activeset.SQPActiveSetMinimizer.Solution
import dev.nm.solver.IterativeSolution

fun problem() : ConstrainedOptimProblem {
    // the objective function: min f(x)
    val f: RealScalarFunction = object : RealScalarFunction {
        override fun evaluate(x: Vector): Double {
            val x1: Double = x.get(1)
            val x2: Double = x.get(2)
            return x1 * x1 + x2
        }

        override fun dimensionOfDomain(): Int {
            return 2
        }

        override fun dimensionOfRange(): Int {
            return 1
        }
    }

    // the set of equality constraints, Ex = 0
    val equal: EqualityConstraints = GeneralEqualityConstraints(
        object : RealScalarFunction {
            override fun evaluate(x: Vector): Double {
                val x1: Double = x.get(1)
                val x2: Double = x.get(2)
                return x1 * x1 + x2 * x2 - 9.0
            } // E

            override fun dimensionOfDomain(): Int {
                return 2
            }

            override fun dimensionOfRange(): Int {
                return 1
            }
        }
    )

    // the set of greater-than constraints, Ax >= b
    val greater = LinearGreaterThanConstraints(
        DenseMatrix(arrayOf(doubleArrayOf(1.0, 0.0), doubleArrayOf(-1.0, 0.0), doubleArrayOf(0.0, 1.0), doubleArrayOf(0.0, -1.0))), // A
        DenseVector(1.0, -5.0, 2.0, -4.0) // b
    )

    // construct the SQP problem
    val problem = ConstrainedOptimProblemImpl1(
        f,
        equal,
        greater.toLessThanConstraints()
    )
    return problem
}


// construct an SQP solver
val solver = SQPActiveSetMinimizer(1e-6, 20)

// solve the SQP problem
val p = problem()
val soln: Solution = solver.solve(p)

// the solution
val x = soln.search(
    DenseVector(5.0, 4.0), // the initial guess for x
    DenseVector(-1.0, -1.0), // the Lagrange multipliers for equality constraints (lambda)
    DenseVector(1.0, 1.0, 1.0, 1.0) // the Lagrange multipliers for greater-than constraints (mu)
)
val fx = p.f().evaluate(x)
println("x = $x, fx = $fx")

x = [1.000000, 2.828427] , fx = 3.8284271247461903
